<a href="https://colab.research.google.com/github/Ashi743/my_projects-/blob/main/snow_depth_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import folium
import geopy
import pandas as pd
from folium.plugins import HeatMap
from geopy.geocoders import Nominatim

In [2]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("eaturner/nycweather")

print("Path to dataset files:", path)


100%|██████████| 152k/152k [00:00<00:00, 487kB/s]

Extracting files...
Path to dataset files: /root/.cache/kagglehub/datasets/eaturner/nycweather/versions/1


In [3]:
import os

# Define the dataset path
dataset_path = '/root/.cache/kagglehub/datasets/eaturner/nycweather/versions/1'

# List all files
files = os.listdir(dataset_path)
print("Files in the dataset directory:", files)


Files in the dataset directory: ['nyc_Jan_Jun_2016_weat.csv']


In [5]:
import pandas as pd

# Load a specific CSV file (adjust filename as needed)
csv_file = os.path.join(dataset_path, 'nyc_Jan_Jun_2016_weat.csv')
data = pd.read_csv(csv_file)

data.head()


,DATE,LATITUDE,LONGITUDE,SNOW,PRCP,SNWD,TAVG,TMAX,TMIN
0,20160101,41.0639,-73.7722,0.0,0.0,-9999.0,-9999,-9999,-9999
1,20160102,41.0639,-73.7722,0.0,0.0,-9999.0,-9999,-9999,-9999
2,20160103,41.0639,-73.7722,0.0,0.0,-9999.0,-9999,-9999,-9999
3,20160104,41.0639,-73.7722,0.0,0.0,-9999.0,-9999,-9999,-9999
4,20160105,41.0639,-73.7722,0.0,0.0,-9999.0,-9999,-9999,-9999


In [6]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 32207 entries, 0 to 32206
Data columns (total 9 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   DATE       32207 non-null  int64  
 1   LATITUDE   32207 non-null  float64
 2   LONGITUDE  32207 non-null  float64
 3   SNOW       32207 non-null  float64
 4   PRCP       32207 non-null  float64
 5   SNWD       32207 non-null  float64
 6   TAVG       32207 non-null  int64  
 7   TMAX       32207 non-null  int64  
 8   TMIN       32207 non-null  int64  
dtypes: float64(5), int64(4)
memory usage: 2.2 MB


In [8]:
data["SNOW"]= pd.to_numeric(data["SNOW"], errors='coerce')
data["PRCP"]= pd.to_numeric(data["PRCP"], errors='coerce')

In [10]:
snow_data_positive= data[data["SNOW"]>0]

In [11]:
min_snow= data["SNOW"].min()
print(min_snow)

-9999.0


In [13]:
max_snow_per_location= snow_data_positive.loc[snow_data_positive.groupby(["LATITUDE", "LONGITUDE"])["SNOW"].idxmax()]
print("\n snow depth in inches per location:")

for index, row in max_snow_per_location.iterrows():
    print(f"Latitude: {row['LATITUDE']}, Longitude: {row['LONGITUDE']}, Snow Depth: {row['SNOW']}")

#lat , long to geolocator

geolocator= Nominatim(user_agent="snow_depth_map")
top_3_snow_locations= max_snow_per_location.nlargest(3, 'SNOW')
print("\n Top 3 snow depth locations:")
for index, row in top_3_snow_locations.iterrows():
    location= geolocator.reverse((row['LATITUDE'], row['LONGITUDE']) ,exactly_one=True)
    address= location.address if location else "Address not found"
    print(f"Latitude: {row['LATITUDE']},  Longitude: {row['LONGITUDE']} ,address: {address},  Snow Depth: {row['SNOW']}")


 snow depth in inches per location:
Latitude: 40.2754, Longitude: -74.006, Snow Depth: 10.0
Latitude: 40.297, Longitude: -74.0015, Snow Depth: 11.0
Latitude: 40.3204, Longitude: -74.1927, Snow Depth: 11.5
Latitude: 40.3622, Longitude: -74.0833, Snow Depth: 12.0
Latitude: 40.3671, Longitude: -73.9979, Snow Depth: 15.5
Latitude: 40.4057, Longitude: -74.2549, Snow Depth: 3.8
Latitude: 40.4562, Longitude: -74.2413, Snow Depth: 13.0
Latitude: 40.4739, Longitude: -74.4, Snow Depth: 1.7
Latitude: 40.5421, Longitude: -74.1476, Snow Depth: 5.0
Latitude: 40.5554, Longitude: -74.274, Snow Depth: 12.0
Latitude: 40.5674, Longitude: -74.1327, Snow Depth: 19.5
Latitude: 40.5759, Longitude: -74.2847, Snow Depth: 20.1
Latitude: 40.6386, Longitude: -73.7622, Snow Depth: 8.3
Latitude: 40.63861, Longitude: -73.76222, Snow Depth: 30.3
Latitude: 40.6588, Longitude: -74.3358, Snow Depth: 8.5
Latitude: 40.6623, Longitude: -73.678, Snow Depth: 17.3
Latitude: 40.66694, Longitude: -74.32306, Snow Depth: 3.0
Lat

In [17]:
heat_data =max_snow_per_location[["LATITUDE" ,"LONGITUDE", "SNOW"]].values.tolist()
map_center= [40.7128, -74.0060]
map_nyc= folium.Map(location=map_center, zoom_start=11)
HeatMap(heat_data ,radius=45).add_to(map_nyc)
map_nyc.save("snow_depth_map.html")
map_nyc